Stochastic Policy Gradient
-------

In [1]:
import gym
import numpy as np
from itertools import count                   
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical

Cài đặt các thông số cho quá trình learning

In [3]:
gamma = 0.99      # Hệ số discount reward
lr = 1e-2         # Learning rate

In [4]:
env = gym.make('CartPole-v0')  # Load môi trường CartPole-v0
env.seed(543)                  # Start state của cartpole là giống nhau trên tất cả các episodes

[543]

## Xây dựng Policy  
Policy là 1 Feedforward neural network có đầu vào là state mà agent nhận được, đầu ra là phân bố xác suất actions mà agent sẽ lựa chọn để thực hiện.
**Architecture:** state(1x4) -> Layer(4x128) -> ReLU -> Layer(128x2) -> Softmax -> actions_distribution(1x2)  
**Optimization:** Adam  
**Loss:** MSE

In [5]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4,128)
        self.affine2 = nn.Linear(128,2)

        self.saved_log_probs = []       # reward và log() sử dụng cho training
        self.rewards = []               # ....
    def forward(self,x):
        x = F.relu(self.affine1(x))
        x = F.softmax(self.affine2(x),dim=1)
        return x
policy = Policy()                                 
optimizer = optim.Adam(policy.parameters(),lr=lr)

Xây dựng hàm lựa chọn action cho agent khi biết đầu vào là state. Policy sẽ dự đoán phân bố xác suất của hành động, sau đó chúng ta sẽ lựa chọn hành động ngẫu nhiên dựa trên phân bố đó, đồng thời lưu trữ giá trị $ \log() $ của xác suất hành động được chọn phục vụ cho việc update Policy

In [6]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)       # numpy -> tensor
    probs = policy(Variable(state))                            # Dự đoán phân bố xác suất của hành động 
    action = np.random.choice([0,1],p=probs.data[0].numpy())   # Lựa chọn hành động ngẫu nhiên dựa vào phân bố trên 
    policy.saved_log_probs.append(torch.log(probs[0][action])) # Lưu trữ log()
    return action

Xây dựng hàm updatePolicy. Sau mỗi episode, chúng ta nhận được 2 chuỗi rewards và saved_log_probs của mỗi action. Chúng ta sẽ tiến hành cập nhật Policy dựa vào công thức:  

$$\theta = \theta + G_t \triangledown_\theta log\pi(a_t|s_t,\theta)$$

Do Policy được thay thế bằng Neural Network nên hàm loss được xác định như sau:

$$loss = - G_t \triangledown_\theta log\pi(a_t|s_t,\theta) $$

$ G_t $ là reward của action được chọn sau khi đã discounted trên toàn bộ episode, được tính theo công thức:

$$G_t = r_t + \gamma G_{t+1}$$

$ r_t $ là reward nhận được ở action t, $ G_{end} $ = $ r_{end} $


In [7]:
def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R               # discount reward
        rewards.insert(0, R)                       
    rewards = torch.Tensor(rewards)                                                      # numpy -> tensor
    rewards = (rewards - rewards.mean()) / (rewards.std() + np.finfo(np.float32).eps)    # rewards normalization
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)                                           # loss caculation
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()      # sum up loss
    policy_loss.backward()                          # backprop
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

## Main

In [ ]:
if __name__ == '__main__':
    for i_episode in count(1):
        if i_episode % 10 == 0: print(i_episode)
        state = env.reset()                                   # start a new game
        for t in range(10000):
            action = select_action(state)                     # select action
            state,reward,done,_ = env.step(action)            # execute action
            env.render()                                      # show GUI
            policy.rewards.append(reward)                     # save reward
            if done:
                break
        finish_episode()                                      # update policy

Kết quả sau khi thực hiện được 60 episodes

![Alt Text](https://media.giphy.com/media/xULW8A8DeiacECjLiw/giphy.gif)